In [1]:
# import packages
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, IrrigationManagement
from aquacrop.utils import prepare_weather, get_filepath
import numpy as np
import matplotlib as mplt
import matplotlib.pyplot as plt
import datetime as dt
import random as random
import pandas as pd
import os

# get current working directory to load and save files
thedir = os.getcwd()

# use cwd to to get folders containing figures and data
datadir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'acrop/model_results'))

The second step is to set the parameters as outlined in exercise 7.7 of the FAO Aquacrop guidebook. See the set of tutorial jupyter notebooks from Thomas Kelley for detailed instructions on how to set up and parameterize the model [here](https://aquacropos.github.io/aquacrop/)

All but one requested paramters were succesfully set, namely the "sprinkler" irrigation method. However, since irrigation in the model is set to occur two times to a "depth of 30cm" and the focus is not the water efficiency of the crop, this differential is not expected to have a significant impact on the results.

```{list-table}
:header-rows: 1

* - Parameter
  - Value
* - Soil
  - Sandy Loam
* - Crop
  - Winter Wheat 'Wheat GDD'
* - Initial Water Content
  - Wilting Point 'WP'
* - Weather
  - tunis_climate
* - Simulation start date
  - 1979/08/15
* - Simulation end date
  - 2002/05/31
```

### Generate random dates within passed intervals
The next code blocks generates random dates between specified intervals and then uses these dates to create the irrigation management file for each run of the Aquacropmodel. The irrigation schedule consists of a date and an irrigation depth, which is fixed at 30 cm.

In [2]:
def get_date(first,last,year):
    """
    function generates a random date in the interval [first, last]
    
    Parameters:
    first1 (str): start of interval of possible watering dates
    last1 (str): end of interval of possible watering dates
    year (int): the year to use
    Returns:
    str: a YYYY-MM-DD str of a date between the provided intervals
    """
    start = dt.date(year, int(first[0:2]), int(first[3:]))
    end = dt.date(year, int(last[0:2]), int(last[3:]))
    interval_time = end - start
    interval_days = interval_time.days
    random_days = random.randrange(interval_days)
    irrig = str(start + dt.timedelta(days = random_days))
    if irrig[5:]=="02-29":
        irrig = get_date(first,last,year)
    return irrig

def genStableCalendar(first1,last1,first2,last2,year,depth):
    """
    function to generate random days for each run of the model but remains constant across growing seasons
    
    Parameters:
    first1 (str): start of first interval of possible watering dates
    last1 (str): end of first interval of possible watering dates
    first2 (str): start of second interval of possible watering dates
    last2 (str): end of second interval of possible watering dates
    year (int): the year to begin
    
    Returns:
    list: returns a list of two elements [Aquacrop-OSPY calendar item,list of dates]
    """
    cal = []
    md = []
    cal.append(F'{year}-12-15')
    first = get_date(first1,last1,year+1)
    second = get_date(first2,last2,year+1)
    day_month = first[5:]
    day_month_ = second[5:]
    cal.append(first)
    cal.append(second)
    md.append([day_month,day_month_])
    for i in range(year+1,2002):
        cal.append(F'{i}-12-15')
        cal.append(F'{i+1}-{day_month}')
        cal.append(F'{i+1}-{day_month_}')
    calendar_pd = pd.DataFrame(cal)
    cal_pd = calendar_pd.set_axis(['Date'], axis=1, inplace=False)
    cal_pd['Depth'] = depth
    # irrigation
    # note that clean water is not specified as it is the default value for irrigation.
    # note that the default irrigitation method is used to achieve the 30 cm)
    im = IrrigationManagement(irrigation_method=3,Schedule = cal_pd,AppEff=100,WetSurf=100)
    return [im,md]

### Define functions to run the aquacrop ospy model
The next codeblock calls the runModel (defined below) function with the specified calender for the specified number of runs.

In [3]:
def runStable(runs,first1,last1,first2,last2,year,depth):
    """
    function runs the stable calendar
    
    Parameters:
    runs (int): the number of runs
    first1 (str): start of first interval of possible watering dates
    last1 (str): end of first interval of possible watering dates
    first2 (str): start of second interval of possible watering dates
    last2 (str): end of second interval of possible watering dates
    year (int): the year to begin
    Returns:
    list: returns a list of two elements [outputs dataframe,list of dates]
    """ 
    i = 1
    mylist = []
    dates= []
    while i <= runs:
        newcal = genStableCalendar(first1,last1,first2,last2,year,depth)
        cal = newcal[0]
        dates.append([newcal[1],i])
        mylist.append(run_model(cal,i))
        if i%10==0:
            print(f'run # {i}')
        i+=1
    new_stable = pd.concat(mylist)
    new_stable.to_csv(F"{datadir}/stable_cal.csv")
    return [new_stable,dates]


This function runs the model and assembles the required outputs. Note that the outputs are divided into four categories: water flux, water storage, crop growth and "final stats".

The model aggregates each variable per season according to the following table.


```{list-table} Variable aggregation
:header-rows: 1

* - Variable included in Aquacrop Model Output
  - Method used to aggregate values across seasons
* - Yield (tonnes/ha)
  - Maximum value within each season
* - Surface Evaporation
  - Cumulative sum for each season
* - Biomass
  - Maximum value within each season
* - Canopy Cover
  - Maximum value within each season
* - Harvest Index
  - Maximum value within each season
```

The function returns a dataframe with these variables appended to the variables from the final statistics from that season.

A no irrigation run_model function is defined as well for later comparison with the results.

In [13]:
def run_model(cal,i):
    """
    function to run model and return results for given panda dataframe of irrigation dates and depths
    Parameters:
    cal (OSPY object): irrigation management object created by OSPY
    
    Returns:
    
    """
    model = AquaCropModel(sim_start_time=simStart,
                      sim_end_time=simEnd,
                      weather_df=weather_data,
                      soil=my_soil,
                      crop=my_crop,
                      initial_water_content=my_iwc,
                      irrigation_management=cal)
    model.run_model(till_termination=True)

    final_stats = model._outputs.final_stats

# assemble water flux variables
    water_flux = model._outputs.water_flux
    wf = water_flux.groupby(["season_counter"]).sum()
    wf = wf[["Es","Tr","DeepPerc","CR"]]
    wf["Season"] = wf.index
    wf.reset_index(inplace = True,drop = True)

# assemble crop growth variables
    crop_growth = model._outputs.crop_growth
    crop_growth["Season"] = crop_growth["season_counter"]
    crops_max = crop_growth.groupby(["season_counter"]).max()[["biomass","harvest_index","biomass_ns","canopy_cover","Season"]]
    
# merge crop growth, water flux and final stats variables into one df
    merged = final_stats.merge(wf,how="inner",on=["Season"])
    outputs = merged.merge(crops_max,how ="inner", on =["Season"])
    outputs["run"] = i
    return outputs

def run_model_no_irrig():
    """
    function to run model and return results with no irrigation schedule
    
    Returns:
    pd df object of results of model run
    """
    model = AquaCropModel(sim_start_time=simStart,
                      sim_end_time=simEnd,
                      weather_df=weather_data,
                      soil=my_soil,
                      crop=my_crop,
                      initial_water_content=my_iwc)
    model.run_model(till_termination=True)
    final_stats = model._outputs.final_stats
    # assemble water flux variables
    water_flux = model._outputs.water_flux
    wf = water_flux.groupby(["season_counter"]).sum()
    wf = wf[["Es","Tr","DeepPerc","CR"]]
    wf["Season"] = wf.index
    wf.reset_index(inplace = True,drop = True)

# assemble crop growth variables
    crop_growth = model._outputs.crop_growth
    crop_growth["Season"] = crop_growth["season_counter"]
    crops_max = crop_growth.groupby(["season_counter"]).max()[["biomass","harvest_index","biomass_ns","canopy_cover","Season"]]
    
# merge crop growth, water flux and final stats variables into one df
    merged = final_stats.merge(wf,how="inner",on=["Season"])
    outputs = merged.merge(crops_max,how ="inner", on =["Season"])
    return outputs

### Output formatting
The output of the model is then organized and general statistics on each of the important variables is collected.

The output is written to a csv file for use in the main book sections.

In [14]:
# assign outputs from stable
def getOutputsPD():
    """
    function to get results pd from results list
    """
    ns_pd = new_stable[0]
    return ns_pd

def getOutputsDate():
    """
    function to get dates pd from results list
    """
    ns_dt = pd.DataFrame(new_stable[1])
    return ns_dt

def getDates(df1,df2):
    """
    function to get dates pd into usefel format
    
    Parameters:
    runs (int): the number of runs
    df1 (str): start of first interval of possible watering dates
    df2 (str): end of first interval of possible watering dates
    
    Returns:
    dataframe of irrigation dates with corresponding run number. 
    """
    # assign outputs from random
    #nr_pd = new_random[0]

    # turn randomized dates into panda dataframes for inclusion in analysis

    # clean-up nested list of lists of lists into useful panda dataframe object
    col_1 = df1.iloc[:,0]
    col_2 = df1.iloc[:,1]
    first = []
    second = []
    run = []
    k = 1
    for i in col_1:
        for j in i:
            first.append(j[0])
            second.append(j[1])
            run.append(k)
            k = k+1
    stable_dates = pd.DataFrame({"run":run,"date_1":first,"date_2":second})

    # create new dataframe of the same length as outputs object for later merging

    first = []
    last = []
    run = []
    for i in df2.run:
        for j in range(len(stable_dates.run)):
            if stable_dates.at[j,"run"] == i:
                first.append(stable_dates.at[j,"date_1"])
                last.append(stable_dates.at[j,"date_2"])
                run.append(i)
    stable_dates_ = pd.DataFrame({"date_1":first,"date_2":last,"run":run})
    return stable_dates

def getStatsRuns(df1,df2):
    """
    function aggregates statistics from the model output across runs.
    
    Parameters:
    ns_pd (df): dataframe of outputs from runs
    Returns:
    list: returns a dataframe of statistics aggregated across runs.
    """
    stats = df1.groupby("run").agg({'Yield (tonne/ha)' : ['mean', 'min','max','median','var'], 
                            'biomass' : ['mean', 'min','max','median','var'],
                            'harvest_index' : ['mean', 'min','max','median','var'],
                            'canopy_cover': ['mean', 'min','max','median','var'],
                            'Es' : ['mean', 'min','max','median','var'],
                            'Tr': ['mean', 'min','max','median','var'],
                            'DeepPerc': ['mean', 'min','max','median','var'],
                            'CR': ['mean', 'min','max','median','var']})
    # flatten multilevel column index using "-" to join levels.
    stats.columns = ["_".join(pair) for pair in stats.columns]

    # save to csv
    

    # merge dates into stats dataframe. Merge on "run" column.
    stats.reset_index(drop = False,inplace = True)
    stats = stats.merge(df2)
    return stats

def getStatsSeasons(df1):
    """
    function aggregates statistics from the model output across seasons
    
    Parameters:
    df1 (df): dataframe of outputs from get_Stats
    """
    stats_2 = df1.groupby("Season").agg({'Yield (tonne/ha)' : ['mean', 'min','max','median','var'], 
                            'biomass' : ['mean', 'min','max','median','var'],
                            'harvest_index' : ['mean', 'min','max','median','var'],
                            'canopy_cover': ['mean', 'min','max','median','var'],
                            'Es' : ['mean', 'min','max','median','var'],
                            'Tr': ['mean', 'min','max','median','var'],
                            'DeepPerc': ['mean', 'min','max','median','var'],
                            'CR': ['mean', 'min','max','median','var']})
    stats_2.columns = ["_".join(pair) for pair in stats_2.columns]
    stats_2.to_csv(F"{datadir}/agg_by_season_{depth}.csv")

def getDays(df):
    """
    function that appeds the number of days since planting for the first and second watering dates to the result of get_Stats()
    
    Parameters:
    df1 (df): dataframe of outputs from get_Stats
    df2 (df): dataframe of dates from getDates
    """
    first = []
    start = dt.date(1979,12,15)    
    for i in df.date_1:
        diff = dt.date(1980,int(i[:2]),int(i[3:])) - start
        first.append(diff)
    second = []
    for i in df.date_2:
        diff = dt.date(1980,int(i[:2]),int(i[3:])) - start
        second.append(diff)
    stats["irrig_1"] = first
    stats["irrig_2"] = second
    stats["irrigation_1"] = df.irrig_1.dt.days
    stats["irrigation_2"] = df.irrig_2.dt.days
    stats.to_csv(F"{datadir}/agg_by_runs_{depth}.csv")

### running the model
Here the model is run by:
1) setting the ```runs_s``` variable equal to the number of runs.
2) uncommenting the ```new_stable``` function
3) uncommenting all the function calls in the next code block.

In [ ]:
# declare parameters
filepath=get_filepath('tunis_climate.txt')
weather_data = prepare_weather(filepath)
my_soil = Soil(soil_type='SandyLoam')
my_crop = Crop('WheatGDD', planting_date='12/01')
my_iwc = InitialWaterContent(value = ["WP"])
simStart = "1979/08/15"
simEnd = "2002/05/31"

noirrig = run_model_no_irrig()
noirrig.to_csv(F'{datadir}/no_irrig.csv')
runs_s = 250
interval1 = ["01-15","03-15"]
interval2 = ["03-16","05-15"]
year = 1979
depth = 5
new_stable = runStable(runs_s,interval1[0],interval1[1],interval2[0],interval2[1],year,depth)

df1 = getOutputsPD()
dates = getOutputsDate()
dates = getDates(dates,df1)
stats = getStatsRuns(df1,dates)
getDays(stats)
getStatsSeasons(df1)